In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
url = 'https://page.kakao.com/menu/10011/screen/84?sort_opt=latest'
options = Options()
options.add_argument("--headless") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome()
driver.get(url)
base_url = 'https://page.kakao.com'
urls = []
for i in range(50):
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

content = soup.select_one('div.flex.w-full.grow.flex-col.px-122pxr')
books = content.select('div.grid > div')
for book in books:
  page_url = base_url+book.find('a')['href']
  urls.append(page_url)

df = pd.DataFrame(urls)
df.to_csv('links.csv', index=False)

driver.quit()

In [3]:
options = Options()
options.add_argument("--headless") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome()
df = pd.read_csv('links_2.csv')

data_list = []

for link in df['link'].iloc[35001:40001]:
  tab_link = link+'?tab_type=about'
  data = {}

  try:
    # 두번째 페이지 먼저 호출
    driver.get(tab_link)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    button = soup.select_one('button > div.line-clamp-1')
    if button:
      driver.execute_script("arguments[0].click();", button)
      time.sleep(1)

    keywords = soup.select('div.flex.w-full.flex-wrap.px-18pxr.pb-10pxr > a > div > span')

    if not keywords:
      print(tab_link)
      continue

    others = soup.select('div.flex.w-full.flex-col.items-start.px-18pxr.pb-18pxr > div')[2:]

    # 첫번째 페이지 호출
    driver.get(link)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.select_one('div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr')

    title = content.select_one('span.font-large3-bold').text
    author = content.select_one('span.font-small2').text
    genre = content.select('span.break-all.align-middle')[1].text
    view = content.select('span.text-el-70.opacity-70')[1].text
    ing = content.select('span.font-small2.text-el-70.opacity-70')[1].text
    uploaded = soup.select('div.flex.h-full.flex-1.items-center.space-x-8pxr > span')[0].text
    date = soup.select_one('div.flex.flex-col.pr-14pxr > div > span.break-all.align-middle').text
    reviewCount = soup.select('div.flex.h-full.flex-1.items-center.space-x-8pxr > span')[1].text

    data = {
      'title' : title,
      'author' : author,
      'genre' : genre,
      'viewCount' : view,
      'ing' : ing,
      'uploaded' : uploaded,
      'date' : date,
      'reviewCount' : reviewCount,
      'keywords' : ''
    }
    data['keywords'] = ''.join(keyword.text for keyword in keywords)
    data['publisher'] = others[0].select('span')[1].text
    data['users'] = others[1].select('span')[1].text
    data['price'] = others[2].select('span')[1].text

    data_list.append(data)
  except Exception as e:
    print("Error by", e)
    continue

# 페이지 종료
driver.quit()

df1 = pd.DataFrame(data_list)
df1.to_csv('data_8.csv', index=False)

https://page.kakao.com/content/64369765?tab_type=about
https://page.kakao.com/content/64354178?tab_type=about
https://page.kakao.com/content/64389188?tab_type=about
https://page.kakao.com/content/64261977?tab_type=about
https://page.kakao.com/content/64261958?tab_type=about
https://page.kakao.com/content/64471603?tab_type=about
https://page.kakao.com/content/64474878?tab_type=about
https://page.kakao.com/content/64471410?tab_type=about
https://page.kakao.com/content/64471213?tab_type=about
https://page.kakao.com/content/64470829?tab_type=about
https://page.kakao.com/content/64480557?tab_type=about
https://page.kakao.com/content/64395981?tab_type=about
https://page.kakao.com/content/64395534?tab_type=about
https://page.kakao.com/content/64393437?tab_type=about
https://page.kakao.com/content/64388988?tab_type=about
https://page.kakao.com/content/64388975?tab_type=about
https://page.kakao.com/content/64370115?tab_type=about
https://page.kakao.com/content/64348644?tab_type=about
https://pa

In [38]:
df = pd.read_csv('test_1.csv')

# 단행본 처리
df = df[~df['title'].apply(lambda x: bool(re.search(r'단행본', x)))]

# viewCount 전처리
def convert_view_count(value):
  if '만' in value:
    value = value.replace('만', '')
    return int(float(value) * 10000)
  else:
    return int(value.replace(',', ''))
  
df['viewCount'] = df['viewCount'].apply(convert_view_count)

# reviewCount 전처리
df['reviewCount'] = df['reviewCount'].str.extract('(\d+)', expand=False).astype(int)

# 연재/완결 전처리
def convert_ing(value):
  if '연재' in value:
    value = value.replace(value, '연재')
    return value
  else:
    return value

df['ing'] = df['ing'].apply(convert_ing)

# 업로드 화 전처리
df['uploaded'] = df['uploaded'].str.extract('(\d+)', expand=False).astype(int)

# users가 00이용가 아닌 것 처리
df.drop(df[df['users'] == '디씨씨이엔티주식회사'].index, inplace=True)

# price 전처리
df['price'] = df['price'].str.extract('(\d+)', expand=False).astype(int)

# date 전처리
df['date'] = pd.to_datetime(df['date'], format='%y.%m.%d')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 0 to 56
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        34 non-null     object        
 1   author       34 non-null     object        
 2   genre        34 non-null     object        
 3   viewCount    34 non-null     int64         
 4   ing          34 non-null     object        
 5   uploaded     34 non-null     int64         
 6   date         34 non-null     datetime64[ns]
 7   reviewCount  34 non-null     int64         
 8   keywords     34 non-null     object        
 9   publisher    34 non-null     object        
 10  users        34 non-null     object        
 11  price        34 non-null     int64         
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 3.5+ KB


In [47]:
df.head()

,title,author,genre,viewCount,ing,uploaded,date,reviewCount,keywords,publisher,users,price
0,퇴장한 악녀의 브런치 카페,지미신,로판,815,연재,80,2024-06-27,3,#로맨스판타지#초월적존재#영혼체인지#왕족/귀족#동거물#오해물#첫사랑#능력남#다정남#...,코핀커뮤니케이션즈,전체이용가,100
1,양치기 소년에게 납치당했습니다,멜리타,로판,25,완결,149,2024-06-27,0,#로맨스판타지#신화물#운명#성장물#다정남#왕족/귀족#대형견남#능력녀#걸크러쉬#계략녀...,다소다미디어,전체이용가,100
2,벤츠 타는 마법사,정주(丁柱),판타지,1016,완결,161,2024-06-27,0,#차원이동#마법사,라온이앤엠,전체이용가,100
3,무한 회귀로 천마가 되다,제허니,판타지,1729,연재,50,2024-06-27,0,#회귀#천재#노력#천마#무공,현나라,전체이용가,100
4,몸 잘 쓰는 천만 배우,이새우,현판,990,완결,175,2024-06-27,0,#성장#노력#배우,로크미디어,전체이용가,100
